### Setup and index
Select a single measurement from database

In [ ]:
import lib.csv_helpers as csv
import lib.data_process 
import lib.json_setup as json_setup
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd

# change directory to wherever you have your setup and data folders
os.chdir('/Users/calum/Desktop/spectrometer')

# select the appropriate setup file, and read it in
setup_path = "setup/default_setup.json"
setup = json_setup.json_to_dict(setup_path)

# Read in the index file of the metadata 
meta_df = csv.read_metadata(setup)

# Filter for the measurement you are interested in
meta_df = csv.select_from_metadata('fluid', 'Air', meta_df)
# meta_df = csv.select_from_metadata('element', 'A01', meta_df)
meta_df

### Process data

Apply any desired processing options (as defined in the data_process.py file)

In [ ]:

# Read in the actual spectral data, merging all measurements into a single dataframe
merged_df, title = csv.merge_dataframes(setup, meta_df)

# Configure the parameters for processing the data
dp = lib.data_process.DataProcessor()
dp.apply_avg_repeats = True
dp.apply_normalise = True
dp.apply_smooth = False
dp.apply_trim = False
dp.apply_interpolate = False
dp.apply_round = True

dp.smooth_points = 3
dp.wavelength_trim_min = 540
dp.wavelength_trim_max = 730
dp.round_decimals = 3

# Apply the processing
df = dp.process_dataframe(merged_df)
df

### Get Stats and export

In [ ]:
export = pd.DataFrame()
for row in meta_df.index:
    stats_df = csv.get_stats_single(setup, dp, meta_df, row, std_deviation=True)

    # Accumulate the dataframes in a large 'export' dataframe
    export = pd.concat([export, stats_df], axis=0)

    export.sort_index(inplace=True)
    export.sort_values(by=['element'], inplace=True, kind="stable")

export

### Custom Processing
An example of creating a derivative function

In [ ]:
# Define a custom function
def derivative(df, trim_nm=100, smooth_points=30):

    for col in df:
        # smooth first
        df[f'{col}_smoothed'] = df[f'{col}'].rolling(window=smooth_points, center=True).mean()

        # Get the literal minimum
        wl_minimum = df[f'{col}_smoothed'].idxmin()

        # trim the derivative at +/- trim_nm around the minimum
        df[f'{col}_smoothed'] = df[f'{col}_smoothed'].loc[df.index >= wl_minimum -trim_nm]
        df[f'{col}_smoothed'] = df[f'{col}_smoothed'].loc[df.index <= wl_minimum +trim_nm]

        # create a wavelength series so the .diff() function on it
        wavelength_series = df.index.to_series()

        # This does a simple delta y / delta x type of derivative
        df[f'{col}_deriv'] = df[f'{col}_smoothed'].diff() / wavelength_series.diff()

    return df

# Apply custom function
df = derivative(df, trim_nm=100, smooth_points=30)

# Get inflections by finding the index (i.e. row numbers) of the min and max of the smoothed derivative
# Then looking up the wavelength at that those indicies.
inflection_min = df['avg_deriv'].idxmin()
inflection_max = df['avg_deriv'].idxmax()
print(f'{inflection_min=}')
print(f'{inflection_max=}')

### Plot

In [ ]:


for col in df.columns:
    if col == 'avg_deriv': #Normalise for better plotting
        maxval = df[col].max() - df[col].min()
        df[col] = df[col] / maxval - df[col].min()

    plt.plot(df.index, df[col], label=col)

plt.xlabel("Wavelength (nm)", fontsize=20)
plt.ylabel("Transmission (%)", fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
fig=plt.gcf()
fig.set_size_inches(15, 8)

plt.axvline(x=inflection_min, label=f"Inflection_min={inflection_min}", color='r')
plt.axvline(x=inflection_max, label=f"Inflection_max={inflection_max}", color='r')
plt.legend()

plt.show()

In [ ]:
lib.data_process.plot(df)